In [1]:
print ("Hello")

Hello


In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
#pip install dataset
# pip install transformers[sentencepiece]
# pip insstall sacrebleu rouge_score /evaluation
# pip install py7zr
# pip install transformers accelerate

In [4]:
device = "cuda" if torch.cuda.is_available()else "cpu"
device

'cuda'

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mesuv\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

d:\gen_ai\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
from datasets import load_dataset, load_from_disk, load_metric

In [8]:
model_ckpt = "google/pegasus-cnn_dailymail"

In [9]:
tokenizer= AutoTokenizer.from_pretrained(model_ckpt) # I am now getting weights for my 

In [10]:
model_pegasus= AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to("cpu") #device=cuda

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
dataset_samsum = load_dataset("samsum")

In [12]:
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [13]:
dataset_samsum["train"]

Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 14732
})

In [14]:
dataset_samsum["train"]["dialogue"]

["Amanda: I baked  cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)",
 'Olivia: Who are you voting for in this election? \r\nOliver: Liberals as always.\r\nOlivia: Me too!!\r\nOliver: Great',
 "Tim: Hi, what's up?\r\nKim: Bad mood tbh, I was going to do lots of stuff but ended up procrastinating\r\nTim: What did you plan on doing?\r\nKim: Oh you know, uni stuff and unfucking my room\r\nKim: Maybe tomorrow I'll move my ass and do everything\r\nKim: We were going to defrost a fridge so instead of shopping I'll eat some defrosted veggies\r\nTim: For doing stuff I recommend Pomodoro technique where u use breaks for doing chores\r\nTim: It really helps\r\nKim: thanks, maybe I'll do that\r\nTim: I also like using post-its in kaban style",
 "Edward: Rachel, I think I'm in ove with Bella..\r\nrachel: Dont say anything else..\r\nEdward: What do you mean??\r\nrachel: Open your fu**ing door.. I'm outside",
 "Sam: hey  overheard rick say something\r\nSam: i don't k

In [15]:
dataset_samsum["train"]["dialogue"][0]

"Amanda: I baked  cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)"

In [16]:
dataset_samsum["train"]["summary"][0]

'Amanda baked cookies and will bring Jerry some tomorrow.'

In [17]:
dataset_samsum["train"]["dialogue"][1]

'Olivia: Who are you voting for in this election? \r\nOliver: Liberals as always.\r\nOlivia: Me too!!\r\nOliver: Great'

In [18]:
dataset_samsum["train"]["summary"][1]

'Olivia and Olivier are voting for liberals in this election. '

In [19]:
for split in dataset_samsum:
    print(split)

train
test
validation


In [20]:
split_length = [len(dataset_samsum[split]) for split in dataset_samsum]
print(f"Split lengths: {split_length}")
print(f"Features: {dataset_samsum['train'].column_names}")
print("\nDialogue:")

print(dataset_samsum["test"][1]["dialogue"])

print("\nSummary:")

print(dataset_samsum["test"][1]["summary"])

Split lengths: [14732, 819, 818]
Features: ['id', 'dialogue', 'summary']

Dialogue:
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I'll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)

Summary:
Eric and Rob are going to watch a stand-up on youtube.


In [21]:
for split in dataset_samsum:
    print(dataset_samsum[split])

Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 14732
})
Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 819
})
Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 818
})


In [22]:
dataset_samsum["train"].column_names

['id', 'dialogue', 'summary']

In [23]:
dataset_samsum["test"]["dialogue"][0]

"Hannah: Hey, do you have Betty's number?\nAmanda: Lemme check\nHannah: <file_gif>\nAmanda: Sorry, can't find it.\nAmanda: Ask Larry\nAmanda: He called her last time we were at the park together\nHannah: I don't know him well\nHannah: <file_gif>\nAmanda: Don't be shy, he's very nice\nHannah: If you say so..\nHannah: I'd rather you texted him\nAmanda: Just text him 🙂\nHannah: Urgh.. Alright\nHannah: Bye\nAmanda: Bye bye"

In [24]:
dataset_samsum["train"]["dialogue"][0]

"Amanda: I baked  cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)"

In [25]:
def convert_examples_to_features(data_in_batch):
    input_encoding=tokenizer(data_in_batch["dialogue"], max_length=1024, truncation= True)

    with tokenizer.as_target_tokenizer():
        target_encoding= tokenizer(data_in_batch["summary"],max_length=128, truncation=True)


    return{
        "input_ids": input_encoding["input_ids"],
        "attention_mask":input_encoding["attention_mask"],
        "labels":target_encoding["input_ids"]
    }

In [26]:
dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features,batched=True)

In [27]:
dataset_samsum_pt=dataset_samsum.map(convert_examples_to_features,batched=True)

In [28]:
dataset_samsum_pt["train"]

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

In [29]:
dataset_samsum_pt["train"]["input_ids"]

[[12195,
  151,
  125,
  7091,
  3659,
  107,
  842,
  119,
  245,
  181,
  152,
  10508,
  151,
  7435,
  147,
  12195,
  151,
  125,
  131,
  267,
  650,
  119,
  3469,
  29344,
  1],
 [18038,
  151,
  2632,
  127,
  119,
  6228,
  118,
  115,
  136,
  2974,
  152,
  10463,
  151,
  35884,
  130,
  329,
  107,
  18038,
  151,
  2587,
  314,
  1242,
  10463,
  151,
  1509,
  1],
 [4776,
  151,
  4451,
  108,
  180,
  131,
  116,
  164,
  152,
  5377,
  151,
  6843,
  4301,
  83678,
  108,
  125,
  140,
  313,
  112,
  171,
  1425,
  113,
  1549,
  155,
  2371,
  164,
  64428,
  4776,
  151,
  463,
  368,
  119,
  511,
  124,
  557,
  152,
  5377,
  151,
  4384,
  119,
  235,
  108,
  18857,
  1549,
  111,
  1596,
  2073,
  56616,
  161,
  418,
  5377,
  151,
  3183,
  3469,
  125,
  131,
  267,
  696,
  161,
  130,
  116,
  111,
  171,
  579,
  5377,
  151,
  184,
  195,
  313,
  112,
  38244,
  114,
  5713,
  167,
  1088,
  113,
  1553,
  125,
  131,
  267,
  1461,
  181,
  38244,
  

In [30]:
dataset_samsum_pt["train"]["input_ids"][0]

[12195,
 151,
 125,
 7091,
 3659,
 107,
 842,
 119,
 245,
 181,
 152,
 10508,
 151,
 7435,
 147,
 12195,
 151,
 125,
 131,
 267,
 650,
 119,
 3469,
 29344,
 1]

In [31]:
#RAG ->mostly for generation
#embedding
#store
#Rag

In [32]:
dataset_samsum_pt["train"]["attention_mask"][0]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [33]:
from transformers import TrainingArguments, Trainer

In [34]:
from transformers import DataCollatorForSeq2Seq

In [35]:
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer,model=model_pegasus)

In [36]:
#fine tuning over here
trainer_args=TrainingArguments(
    output_dir='pegasus-samsum', num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,weight_decay=0.01,
    logging_steps=10,
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
)

In [37]:
trainer=Trainer(model=model_pegasus,args=trainer_args,
                 tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                train_dataset= dataset_samsum_pt["test"],
                eval_dataset=dataset_samsum_pt["validation"])

d:\gen_ai\venv\Lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [38]:
torch.cuda.empty_cache() 
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "caching_allocator"

In [39]:
trainer.train()

 20%|█▉        | 10/51 [08:28<34:50, 51.00s/it]

{'loss': 3.1157, 'grad_norm': 120.20783996582031, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.2}


 39%|███▉      | 20/51 [15:13<20:40, 40.01s/it]

{'loss': 3.2325, 'grad_norm': 13.664416313171387, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.39}


 59%|█████▉    | 30/51 [22:03<14:29, 41.43s/it]

...

 98%|█████████▊| 50/51 [48:29<00:53, 53.21s/it] 

{'loss': 2.9197, 'grad_norm': 11.859108924865723, 'learning_rate': 5e-06, 'epoch': 0.98}


100%|██████████| 51/51 [49:06<00:00, 57.78s/it]

{'train_runtime': 2946.6559, 'train_samples_per_second': 0.278, 'train_steps_per_second': 0.017, 'train_loss': 3.0382910335765168, 'epoch': 1.0}


TrainOutput(global_step=51, training_loss=3.0382910335765168, metrics={'train_runtime': 2946.6559, 'train_samples_per_second': 0.278, 'train_steps_per_second': 0.017, 'train_loss': 3.0382910335765168, 'epoch': 1.0})

In [40]:
model_pegasus.save_pretrained("pegasus-samsum-model")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


In [41]:
tokenizer.save_pretrained("samsum-tokenizer")

('samsum-tokenizer\\tokenizer_config.json',
 'samsum-tokenizer\\special_tokens_map.json',
 'samsum-tokenizer\\spiece.model',
 'samsum-tokenizer\\added_tokens.json',
 'samsum-tokenizer\\tokenizer.json')

In [42]:
tokenizer=AutoTokenizer.from_pretrained("samsum-tokenizer")

In [44]:
model_I_trained=AutoModelForSeq2SeqLM.from_pretrained("pegasus-samsum-model").to(device)

In [48]:
sample_text_data_for_test = dataset_samsum_pt["train"]["dialogue"][2]
print(sample_text_data_for_test)

Tim: Hi, what's up?
Kim: Bad mood tbh, I was going to do lots of stuff but ended up procrastinating
Tim: What did you plan on doing?
Kim: Oh you know, uni stuff and unfucking my room
Kim: Maybe tomorrow I'll move my ass and do everything
Kim: We were going to defrost a fridge so instead of shopping I'll eat some defrosted veggies
Tim: For doing stuff I recommend Pomodoro technique where u use breaks for doing chores
Tim: It really helps
Kim: thanks, maybe I'll do that
Tim: I also like using post-its in kaban style


In [49]:
summary=dataset_samsum_pt["train"]["summary"][2]
summary

'Kim may try the pomodoro technique recommended by Tim to get more stuff done.'

In [50]:
pipeline_object=pipeline("summarization",model=model_I_trained,tokenizer=tokenizer)

In [52]:
pipeline_object(sample_text_data_for_test)[0]["summary_text"]

"Kim was going to do lots of stuff but ended up procrastinating .<n>We were going to defrost a fridge so instead of shopping I'll eat some defrosted veggies .<n>For doing stuff I recommend Pomodoro technique where u use breaks for doing chores ."

In [57]:
predection=pipeline_object(sample_text_data_for_test)[0]["summary_text"]
print("Predected summary\n",predection)

Predected summary
 Kim was going to do lots of stuff but ended up procrastinating .<n>We were going to defrost a fridge so instead of shopping I'll eat some defrosted veggies .<n>For doing stuff I recommend Pomodoro technique where u use breaks for doing chores .


In [56]:
print("Real summary\n",summary)

Real summary
 Kim may try the pomodoro technique recommended by Tim to get more stuff done.


Not good summary so want to fine tune

In [58]:
gen_kwargs={"length_penalty":0.8,"num_beams":8, "max_length":128}

In [63]:
prediction = pipeline_object(sample_text_data_for_test,**gen_kwargs)[0]["summary_text"]


In [64]:
print(prediction)

Kim was going to do lots of stuff but ended up procrastinating .<n>We were going to defrost a fridge so instead of shopping I'll eat some defrosted veggies .<n>For doing stuff I recommend Pomodoro technique where u use breaks for doing chores .


In [66]:
summary

'Kim may try the pomodoro technique recommended by Tim to get more stuff done.'

In [67]:
def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]



def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

        # Finally, we decode the generated texts,
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score

In [70]:
rouge_names=["rouge1","rouge2","rougeL","rougeLsum"]
rouge_metric=load_metric('rouge')

d:\gen_ai\venv\Lib\site-packages\datasets\load.py:756: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [71]:
score = calculate_metric_on_test_ds(
    dataset_samsum['test'][0:10], rouge_metric, trainer.model, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
)

rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = [f'pegasus'] )

100%|██████████| 5/5 [06:09<00:00, 73.95s/it]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.022083,0.0,0.021681,0.021755


In [72]:
## Save model
model_pegasus.save_pretrained("pegasus-samsum-model1")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


In [73]:
## Save tokenizer
tokenizer.save_pretrained("samsum-tokenizer1")

('samsum-tokenizer1\\tokenizer_config.json',
 'samsum-tokenizer1\\special_tokens_map.json',
 'samsum-tokenizer1\\spiece.model',
 'samsum-tokenizer1\\added_tokens.json',
 'samsum-tokenizer1\\tokenizer.json')

In [74]:
tokenizer = AutoTokenizer.from_pretrained("samsum-tokenizer1")

In [75]:
#Prediction

gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}



sample_text = dataset_samsum["test"][0]["dialogue"]

reference = dataset_samsum["test"][0]["summary"]

pipe = pipeline("summarization", model="pegasus-samsum-model1",tokenizer=tokenizer)

##
print("Dialogue:")
print(sample_text)


print("\nReference Summary:")
print(reference)


print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

Your max_length is set to 128, but your input_length is only 122. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)


Dialogue:
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye

Reference Summary:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.

Model Summary:
Amanda: Ask Larry Amanda: He called her last time we were at the park together .<n>Hannah: I'd rather you texted him .<n>Amanda: Just text him .
